In [ ]:
import requests

url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
response = requests.get(url)
data = response.json()
apps = data["applist"]["apps"]
data

In [ ]:
appid = 150
details_url = f"https://store.steampowered.com/api/appdetails?appids={appid}"
details_resp = requests.get(details_url)
details_data = details_resp.json()

print(details_data)
if details_data[str(appid)]['success']:
    app_info = details_data[str(appid)]["data"]
    if app_info.get('type') == 'game':
        print('게임입니다')
    else : 
        print('제외대상입니다')

In [7]:
import pandas as pd
import requests
import time
import os

# --- 설정 ---
OUTPUT_CSV = "스팀게임파일.csv"
FAIL_LOG = "fail_log.txt"
API_DELAY_SEC = 1

# --- Steam Storefront API에서 데이터 가져오기 ---
def fetch_steam_data(appid, delay=API_DELAY_SEC):
    url = f"https://store.steampowered.com/api/appdetails?appids={appid}&cc=kr&l=koreana"
    try:
        resp = requests.get(url, timeout=10)
        time.sleep(delay)

        if resp.status_code != 200:
            print(f"[ERROR] HTTP {resp.status_code} for appid {appid}")
            return None

        data = resp.json()
        if not data.get(str(appid), {}).get('success'):
            print(f"[ERROR] No success flag for appid {appid}")
            return None

        app_data = data[str(appid)]['data']
        if app_data.get('type') != 'game':
            print(f"[SKIP] Not a game: {appid}")
            return None

        categories = [c['description'] for c in app_data.get('categories', [])]
        genres = [g['description'] for g in app_data.get('genres', [])]
        screenshots = [s['path_full'] for s in app_data.get('screenshots', [])]

        return {
            'appid': str(appid),
            'name_kr': str(app_data.get('name', '')),
            'genres': ', '.join(genres),
            'release_date': app_data.get('release_date', {}).get('date', ''),
            'is_coming_soon': str(app_data.get('release_date', {}).get('coming_soon', False)),
            'initial_price': str(app_data.get('price_overview', {}).get('initial', 0) / 100),
            'description_kr': app_data.get('short_description', ''),
            'category': '|'.join(categories) if categories else '',
            'new_description': app_data.get('detailed_description', ''),
            'screenshots': '|'.join(screenshots) if screenshots else ''
        }

    except Exception as e:
        print(f"[EXCEPTION] appid {appid}: {e}")
        return None

# --- 기존 저장된 AppID 불러오기 ---
def load_existing_appids():
    if os.path.exists(OUTPUT_CSV):
        df = pd.read_csv(OUTPUT_CSV)
        df = df[df['appid'].astype(str).str.isdigit()]
        return set(df['appid'].astype(str))
    return set()

# --- 실패한 AppID 불러오기 ---
def load_failed_appids():
    failed = set()
    if os.path.exists(FAIL_LOG):
        with open(FAIL_LOG, 'r') as f:
            for line in f:
                line = line.strip()
                if line.isdigit():
                    failed.add(line)
    return failed

# --- 실패 로그 기록 ---
def append_to_fail_log(appid):
    with open(FAIL_LOG, 'a') as f:
        f.write(f"{appid}\n")

# --- 메인 로직 ---
def main():
    print("🔄 전체 AppID 목록 불러오는 중...")
    app_list_url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
    resp = requests.get(app_list_url)
    apps = resp.json()["applist"]["apps"]

    print(f"✅ 전체 App 수: {len(apps)}")

    existing_ids = load_existing_appids()
    failed_ids = load_failed_appids()

    print(f"✅ 이미 저장된 AppID 수: {len(existing_ids)}")
    print(f"✅ 실패 기록된 AppID 수: {len(failed_ids)}")

    to_process = [
        app for app in apps
        if str(app['appid']) not in existing_ids
        and str(app['appid']) not in failed_ids
    ]

    print(f"🚀 새로 처리할 AppID 수: {len(to_process)}")

    for app in to_process:
        appid = app['appid']
        print(f"\n▶️ Processing {appid} - {app['name']}")

        result = fetch_steam_data(appid)
        if result is None:
            append_to_fail_log(appid)
            continue

        df = pd.DataFrame([result])
        if os.path.exists(OUTPUT_CSV):
            df.to_csv(OUTPUT_CSV, mode='a', header=False, index=False)
        else:
            df.to_csv(OUTPUT_CSV, mode='w', header=True, index=False)

        print(f"[SAVED] AppID {appid}")

    print("\n🎉 모든 AppID 처리 완료!")

# --- 실행 ---
if __name__ == "__main__":
    main()


🔄 전체 AppID 목록 불러오는 중...
✅ 전체 App 수: 256841
✅ 이미 저장된 AppID 수: 7278
✅ 실패 기록된 AppID 수: 25416
🚀 새로 처리할 AppID 수: 224147

▶️ Processing 576950 - Westard
[SAVED] AppID 576950

▶️ Processing 576960 - 🧠 OUT OF THE BOX
[SAVED] AppID 576960

▶️ Processing 576970 - Out Of The Box Demo
[ERROR] No success flag for appid 576970

▶️ Processing 576980 - Western 1849 Reloaded
[SAVED] AppID 576980

▶️ Processing 576990 - My Friend's Rubber Ducky
[SKIP] Not a game: 576990

▶️ Processing 577000 - Witchcraft
[SAVED] AppID 577000

▶️ Processing 577010 - Game of Aces
[SKIP] Not a game: 577010

▶️ Processing 577020 - Planes, Bullets and Vodka: Soundtrack
[SKIP] Not a game: 577020

▶️ Processing 577030 - The Tenth Line - Digital Art Book + Soundtrack
[SKIP] Not a game: 577030

▶️ Processing 577080 - A Dump in the Dark
[SAVED] AppID 577080

▶️ Processing 577100 - Boundel Soundtrack by James Thorley
[SKIP] Not a game: 577100

▶️ Processing 577110 - Age of Fear: Concept Art Book
[SKIP] Not a game: 577110

▶️ Proce

KeyboardInterrupt: 